<i> War is the continuation of politics by other means. </i>
    
    Karl von Clausewitz

Before delving into CoW, we will create a Markov model that captures many of the major dynamics in an interstate alliance system.

In a Markov model, current system state is determined only by previous state combined with a set of transition probabilities and update rules. In an NxN country system, we care about the following elements of state:

- War: Whether two countries are at war.
- Affinity: The military dynamic between two countries, including aid and alliance.
- Power: Relative power dynamics of each country pair.
- Events: Exogenous events provoking conflict.

We will discretize as follows:

- War: {0, 1}; 0 is war and 1 is peace.
- Affinity: {-1, -.5, 0, .5, 1}; 1 represents full opposition, 0.5 negative indirect activity (arms shipments, political pressure, or other opposition), 0 no interaction, -0.5 positive indirect activity, and -1 full alliance.
- Power: {-1, -.9, ..., 1}; -1 is minimum relative power (capitulation) and 1 is maximum power.
- Events: {-1, -.9, ..., 1}; -1 is a maximally tension-lowering event, 0 is no event, and 1 is a maximally tension-raising event.

We also want to capture elements of geographical location and policy through the following fixed variables:

- Proximity: {0, .25, .5, .75, 1}; 0 for no potential interaction fronts and 1 for neighboring countries.
- Aggression: {-1, -.5, 0, .5, 1}; -1 for peaceful orientation and 1 for aggressive orientation.

Whether two countries are at war will depend on the following elements of the prior state:

- Prior war status of self and allies
- Current affinity
- Event influence
- Base transition probabilities and aggression levels

Affinity is affected by the following:

- Prior war status
- Prior affinity
- Prior relative power
- Base transition probabilities

Power is affected by the following:

- Prior power
- Prior power- and proximity-adjusted influence of allies and enemies

We will begin by modeling with a "season" (3-month interval) as the timestep.

Therefore:

    war(c1, c2, t): f(
        war(c1, c2, t-1),
        [war(c2, c3, t-1) * affinity(c1, c3, t) for c3 != c1,c2],
        [war(c1, c3, t-1) * affinity(c2, c3, t) for c3 != c1,c2],
        affinity(c1, c2, t-1),
        event(c1, c2, t-1),
        power(c1, t-1),
        power(c2, t-1),
        aggression(c1),
        aggression(c2)
    )
        
    affinity(c1, c2, t): f(
        war(c1, c2, t-1),
        affinity(c1, c2, t-1),
        power(c1, t-1),
        power(c2, t-1)
    )
    
    power(c1, t): f(
        power(c1, t-1),
        [affinity(c1, c2, t-1) * power(c2, t-1) * proximity(c1, c2) for c2 != c1]
    )

Our plan of attack:

1. Encode update rules and state machine
2. Set base parameters and transition probabilities
3. Simulate and evaluate results for WWI
4. Estimate true parameters and probabilities from CoW
5. Simulate and evaluate results for WWI
6. Experiment with other international systems

In [28]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
import seaborn as sns

In [37]:
# Order: Austria (Austria-Hungary), England (Britain), France, Germany, Italy, Russia, Turkey (Ottoman Empire)

def countries():
    return ['A', 'E', 'F', 'G', 'I', 'R', 'T']

def index():
    return {c: i for i, c in enumerate(countries)}

N = len(war_state)

def war_init():
    return np.zeros((N, N))

def affinity_init():
    return np.array([
        #   A,   E,   F,   G,   I,   R,   T
        [   0,   0,   0,   1,   1,   0,   0],  # A
        [   0,   0,  .5,   0,   0,  .5,   0],  # E
        [   0,  .5,   0,   0,   0,   1,   0],  # F
        [   1,   0,   0,   0,   1,   0,   1],  # G
        [   1,   0,   0,   1,   0,   0,   0],  # I
        [   0,  .5,   1,   0,   0,   0,   0],  # R
        [   0,   0,   0,   1,   0,   0,   0]   # T
    ])

def power_init():
    #                  A,   E,   F,   G,   I,   R,   T
    return np.array([-.2,  .4,  .2,  .6, -.2, -.2, -.2])

def aggression():
    #                  A,   E,   F,   G,   I,   R,   T
    return np.array([  0,   0,   0,   0,   0,   0,   0])

def proximity():
    return np.array([
        #   A,   E,   F,   G,   I,   R,   T
        [   0,  .5,  .5,   1,   1,   1,   1],  # A
        [  .5,   0,   1,   1,  .5,  .5, .25],  # E
        [  .5,   1,   0,   1,   1,  .5, .25],  # F
        [   1,   1,   1,   0,   1,   1,  .5],  # G
        [   1,  .5,   1,   1,   0,  .5,  .5],  # I
        [   1,  .5,  .5,   1,  .5,   0,   1],  # R
        [   1, .25, .25,  .5,  .5,   1,   0]   # T
    ])

def balkans_event():
    return np.array([
        #   A,   E,   F,   G,   I,   R,   T
        [   0,   0,   0,   0,   0,  .8,   0],  # A
        [   0,   0,   0,   0,   0,   0,   0],  # E
        [   0,   0,   0,   0,   0,   0,   0],  # F
        [   0,   0,   0,   0,   0,   0,   0],  # G
        [   0,   0,   0,   0,   0,   0,   0],  # I
        [  .8,   0,   0,   0,   0,   0,  .5],  # R
        [   0,   0,   0,   0,   0,  .5,   0],  # T
    ])

def no_event():
    return np.zeros((N, N))


# Base transitions: (p, increment)

def base_war_trans():
    #                     up,     same,     down
    return np.array([(.05, 1), (.75, 0), (.2, -1)])

def base_affinity_trans():
    #                   down 2,      down 1,      same,       up 1,      up 2
    return np.array([(.022, -1), (.137, -.5), (.682, 0), (.137, .5), (.022, 1)])

def change(base):
    return np.random.choice([e[1] for e in base], 1, p=[e[0] for e in base])[0]

In [38]:
# Rounds to given precision, i.e. step size.
def round_p(n, precision = 1):
    return np.round(n / precision) * precision

# Discretizes by model variable type.
def disc(n, var):
    precisions = {'w': 1, 'a': .5, 'p': .1}
    return round_p(n, precisions[var])

def bound(n, var):
    mins = {'w': 0, 'a': -1, 'p': -1}
    return min(1, max(mins[var], n))

In [39]:
base_war = base_war_trans()
base_aff = base_affinity_trans()
eve_war = 0.7   # event                   -> war coefficient
agg_war = 0.1   # aggression              -> war coefficient
pow_war = 0.2   # power                   -> war coefficient
pow_aff = 0.05  # power                   -> affinity coefficient
pow_pow = 0.5   # power of allies/enemies -> own power coefficient

def allied(c1, c2, aff):
    return aff[c1][c2] > .99

def self_or_allies_war(c1, c2, war0, aff0):
    if war0[c1][c2]:
        return 1.0
    elif any([True for c3 in range(N) if war0[c2][c3] and allied(c1, c3, aff0)]) or \
         any([True for c3 in range(N) if war0[c1][c3] and allied(c2, c3, aff0)]):
        return 1.0
    return 0.0

def war(c1, c2, war0, aff0, pow0, agg, event):
    war1 = self_or_allies_war(c1, c2, war0, aff0)
    war1 += change(base_war)
    war1 += pow_war * np.abs(pow0[c1] - pow0[c2])
    war1 += agg_war * (agg[c1] + agg[c2])
    war1 += eve_war * event[c1][c2]
    return bound(disc(war1, 'w'), 'w')

def affinity(c1, c2, war0, aff0, pow0):
    aff1 = aff0[c1][c2] + pow_aff * (pow0[c1] - pow0[c2]) + change(base_aff)
    return bound(disc(aff1, 'a'), 'a')

def participants(c1, war0):
    # for each country, determine whether to include
    # if included, add power and proximity
    deg1 = [war0[c1][c2] for c2 in range(N)]  # enemies
    deg2 = [any([war0[c2][c3] > 0 and deg1[c2] > 0 for c2 in range(N)]) for c3 in range(N)]  # enemies of enemies
    return [1 if deg1[i] or deg2[i] else 0 for i in range(N)]

def power(c1, war0, aff0, pow0, prox):
    # only update power in war conditions for participants
    parties = participants(c1, war0)
    pow1 = pow0[c1] 
    for c2 in range(N):
        pow1 += pow_pow * np.sum([parties[c2] * aff0[c1][c2] * pow0[c2] * prox[c1][c2]])
    return bound(disc(pow1, 'p'), 'p')

power(0, war_state, aff_state, pow_state, prox)

-0.2

In [43]:
from itertools import product

rounds = 100
prox = proximity()
agg = aggression()
balk = balkans_event()
none = no_event()

w, a, p = war_init(), affinity_init(), power_init()
for i in range(rounds):
    print('------------------------------------------------------------------------------------------')
    print('ROUND {}'.format(i))
    print('------------------------------------------------------------------------------------------')
    cm = sns.light_palette("red", as_cmap=True)
    display(pd.DataFrame(w).style.set_precision(2).background_gradient(cmap=cm))

    cm = sns.light_palette("blue", as_cmap=True)
    display(pd.DataFrame(a).style.set_precision(2).background_gradient(cmap=cm))

    cm = sns.light_palette("green", as_cmap=True)
    display(pd.DataFrame(p).style.set_precision(2).background_gradient(cmap=cm))
    w1, a1, p1 = np.zeros((N, N)), np.zeros((N, N)), np.zeros((N))
    for c1, c2 in product(range(N), repeat=2):
        if c1 == c2:
            p1[c1] = power(c1, w, a, p, prox)
        else:
            w1[c1][c2] = w1[c2][c1] = war(c1, c2, w, a, p, agg, none)
            a1[c1][c2] = a1[c2][c1] = affinity(c1, c2, w, a, p)
    w, a, p = w1, a1, p1
            

------------------------------------------------------------------------------------------
ROUND 0
------------------------------------------------------------------------------------------


,0,1,2,3,4,5,6
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00


,0,1,2,3,4,5,6
0,0.00,0.00,0.00,1.00,1.00,0.00,0.00
1,0.00,0.00,0.50,0.00,0.00,0.50,0.00
2,0.00,0.50,0.00,0.00,0.00,1.00,0.00
3,1.00,0.00,0.00,0.00,1.00,0.00,1.00
4,1.00,0.00,0.00,1.00,0.00,0.00,0.00
5,0.00,0.50,1.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,1.00,0.00,0.00,0.00


,0
0,-0.20
1,0.40
2,0.20
3,0.60
4,-0.20
5,-0.20
6,-0.20


------------------------------------------------------------------------------------------
ROUND 1
------------------------------------------------------------------------------------------


...

,0
0,1.00
1,1.00
2,1.00
3,-1.00
4,1.00
5,1.00
6,1.00


------------------------------------------------------------------------------------------
ROUND 99
------------------------------------------------------------------------------------------


,0,1,2,3,4,5,6
0,0.00,1.00,1.00,1.00,1.00,1.00,1.00
1,1.00,0.00,1.00,1.00,1.00,0.00,1.00
2,1.00,1.00,0.00,1.00,1.00,1.00,1.00
3,1.00,1.00,1.00,0.00,1.00,1.00,1.00
4,1.00,1.00,1.00,1.00,0.00,1.00,1.00
5,1.00,0.00,1.00,1.00,1.00,0.00,1.00
6,1.00,1.00,1.00,1.00,1.00,1.00,0.00


,0,1,2,3,4,5,6
0,0.00,-0.50,1.00,-0.50,1.00,1.00,1.00
1,-0.50,0.00,0.00,-0.50,1.00,0.00,0.50
2,1.00,0.00,0.00,0.50,-0.50,1.00,0.50
3,-0.50,-0.50,0.50,0.00,0.50,-0.50,1.00
4,1.00,1.00,-0.50,0.50,0.00,1.00,-1.00
5,1.00,0.00,1.00,-0.50,1.00,0.00,0.50
6,1.00,0.50,0.50,1.00,-1.00,0.50,0.00


,0
0,1.00
1,1.00
2,1.00
3,-1.00
4,1.00
5,1.00
6,1.00
